<a href="https://colab.research.google.com/github/rafiafsan/LSTM-Long-Short-Term-Memory-/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk

import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Load your CSV (adjust path if necessary)
df = pd.read_csv('/content/test.csv',encoding='latin1')  # uploaded file path


In [ ]:
# cleaning data

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
def preprocess_text(text):

    if not isinstance(text, str):

        return ''

    text = re.sub(r'[^a-zA-Z\s]', '', text)

    text = text.lower()

    tokens = [word for word in text.split() if word not in stop_words]

    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df.dropna(inplace=True)

In [ ]:
# If sentiments are strings like 'positive', 'neutral', 'negative'
sentiment_map = {
    'negative': 0,
    'neutral': 1,
    'positive': 2
}

# Map sentiments to numerical values, handling unknown values
df['sentiment'] = df['sentiment'].map(sentiment_map).fillna(-1).astype(int)

# Check if the dataframe is empty before proceeding
if df[df['sentiment'] != -1].empty:
    print("Warning: DataFrame is empty after filtering. Check your sentiment mapping or data.")
else:
    df = df[df['sentiment'] != -1]

In [ ]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

df['clean_text'] = df['text'].apply(preprocess)

In [ ]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['clean_text'])

sequences = tokenizer.texts_to_sequences(df['clean_text'])
padded = pad_sequences(sequences, maxlen=100)

X = padded
y = to_categorical(df['sentiment'], num_classes=3)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [91]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=64, input_length=100))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))  # 3 classes

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [92]:
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 10s 79ms/step - accuracy: 0.3960 - loss: 1.0824 - val_accuracy: 0.4668 - val_loss: 1.0161
Epoch 2/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.6134 - loss: 0.8726 - val_accuracy: 0.6365 - val_loss: 0.8190
Epoch 3/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step - accuracy: 0.8097 - loss: 0.5141 - val_accuracy: 0.6591 - val_loss: 0.8233
Epoch 4/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.9059 - loss: 0.2786 - val_accuracy: 0.6421 - val_loss: 0.9536
Epoch 5/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - accuracy: 0.9347 - loss: 0.2097 - val_accuracy: 0.6280 - val_loss: 1.0079


In [93]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6524 - loss: 1.0112
Test Accuracy: 62.80%


In [ ]:
def predict_sentiment(text):
    text = preprocess(text)
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=100)
    pred = model.predict(pad)
    label = np.argmax(pred)

    if label == 0:
        print("Predicted Sentiment: Negative 😞 ")
    elif label == 1:
        print("Predicted Sentiment: Neutral 😐")
    else:
        print("Predicted Sentiment: Positive 😊")

# Example input
user_input = input("Enter a sentence: ")
predict_sentiment(user_input)
